In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
dataset = pd.read_csv('output.csv', index_col=0, parse_dates=True)


In [3]:
dataset = dataset[['Oil price', 'S&P500', 'Gold price', 'VN Index']]

In [4]:
duration = int(1)
start_date = datetime.strptime(
                    '2022-06-24',
                    '%Y-%m-%d')

In [5]:
dataset.tail(20)

,Oil price,S&P500,Gold price,VN Index
2022-05-12,110.211250,4039.9512,1823.3251,1205.0088
2022-05-13,110.490000,4023.8900,1814.8000,1182.7700
2022-05-14,111.726660,4018.5967,1816.6667,1179.1633
2022-05-15,112.963330,4013.3032,1818.5333,1175.5566
2022-05-16,114.200000,4008.0100,1820.4000,1171.9500
2022-05-17,112.400000,4088.8500,1825.4000,1228.3700
2022-05-18,109.590000,3923.6800,1822.4000,1240.7600
2022-05-19,112.210000,3900.7900,1847.8000,1241.6400
2022-05-20,113.230000,3901.3600,1848.4000,1240.7100
2022-05-21,112.250000,3925.4900,1850.2334,1233.4100


In [6]:
type(dataset['VN Index'].values)

numpy.ndarray

In [7]:
start_date = datetime(2022, 5, 3)
lags = 6
duration = 1

In [8]:
model_start_date = start_date - timedelta(days=lags+2)
model_end_date = start_date 

start_index = dataset.index.searchsorted(model_start_date)
end_index = dataset.index.searchsorted(model_end_date)

df_train = dataset[start_index:end_index]

In [9]:
df_train

,Oil price,S&P500,Gold price,VN Index
2022-04-25,98.540000,4296.1200,1903.00,1310.9200
2022-04-26,101.700000,4175.2000,1911.10,1341.3400
2022-04-27,102.020000,4183.9600,1895.60,1353.7700
2022-04-28,105.360000,4287.5000,1898.30,1350.9900
2022-04-29,104.690000,4131.9300,1918.60,1366.8000
2022-04-30,91.270004,4416.2550,1875.05,1444.6500
2022-05-01,77.850000,4700.5800,1831.50,1522.5000
2022-05-02,85.886665,4642.0933,1865.50,1521.6766


In [10]:
lags = 6
duration = 1
    # start_index = random.randint(lag_order+1, len(origin_df) - nobs-1)
test_start_index = dataset.index.searchsorted(start_date)
test_end_index = dataset.index.searchsorted(start_date+timedelta(days=duration))
df_test = dataset[test_start_index:test_end_index]

In [11]:
start_date

datetime.datetime(2022, 5, 3, 0, 0)

In [12]:
df_test

,Oil price,S&P500,Gold price,VN Index
2022-05-03,93.92333,4583.607,1899.5,1520.8534


In [13]:
import pickle
model = pickle.load(open('VAR_vnindex.pkl', "rb"))


In [22]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc


In [23]:
diff_df = dataset.diff().dropna()

diff_start_index = diff_df.index.searchsorted(model_start_date)
diff_end_index = diff_df.index.searchsorted(model_end_date)

forecast_input = diff_df.values[diff_start_index:diff_end_index]

fc = model.forecast(y=forecast_input, steps=duration)
df_forecast = pd.DataFrame(fc, index=df_test.index, columns=dataset.columns + '_1d')
df_results = invert_transformation(df_train, df_forecast, second_diff=False)
df_results = df_results[['Oil price', 'S&P500', 'Gold price', 'VN Index']]

frame = [df_train, df_results]
full_result = pd.concat(frame)

In [17]:
df_forecast

,Oil price_1d,S&P500_1d,Gold price_1d,VN Index_1d
2022-05-03,4.339007,-1.455435,48.943314,-7.189912


In [27]:
dataset[['VN Index']].tail(30)

,VN Index
2022-05-02,1521.6766
2022-05-03,1520.8534
2022-05-04,1520.0300
2022-05-05,1360.6800
2022-05-06,1338.4413
2022-05-07,1316.2025
2022-05-08,1293.9637
2022-05-09,1271.7251
2022-05-10,1249.4863
2022-05-11,1227.2476


In [18]:
df_results

,Oil price_1d,S&P500_1d,Gold price_1d,VN Index_1d,Oil price_forecast,S&P500_forecast,Gold price_forecast,VN Index_forecast
2022-05-03,4.339007,-1.455435,48.943314,-7.189912,90.225672,4640.637865,1914.443314,1514.486688


In [24]:
full_result

,Oil price,S&P500,Gold price,VN Index
2022-04-25,98.540000,4296.120000,1903.000000,1310.920000
2022-04-26,101.700000,4175.200000,1911.100000,1341.340000
2022-04-27,102.020000,4183.960000,1895.600000,1353.770000
2022-04-28,105.360000,4287.500000,1898.300000,1350.990000
2022-04-29,104.690000,4131.930000,1918.600000,1366.800000
2022-04-30,91.270004,4416.255000,1875.050000,1444.650000
2022-05-01,77.850000,4700.580000,1831.500000,1522.500000
2022-05-02,85.886665,4642.093300,1865.500000,1521.676600
2022-05-03,90.225672,4640.637865,1914.443314,1514.486688
